# Web Scraping Assignment-3
# C Soma Sekhar
# Batch: DS2307

In [78]:
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import requests
import re
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait


# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars. 

In [86]:
#Get the chrome drivers
driver = webdriver.Chrome()
driver.get('https://www.amazon.in/')

time.sleep(5)

# Find the input element
input_element = driver.find_element(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input')

# Get user input
user_input= input("Enter the product: ")

# Send the input to the input element
input_element.send_keys(user_input)

#search for the product.
search = driver.find_element(By.XPATH, '/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')
search.click()


Enter the product: guitar


# Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [87]:
# Create an empty list to store the URLs
urls = []

# Define the number of pages you want to scrape (adjust as needed)
num_pages_to_scrape = 3

for _ in range(num_pages_to_scrape):
    # Find all the links on the current page
    page_links = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    
    # Extract the URLs from the links and add them to the 'urls' list
    for link in page_links:
        url = link.get_attribute("href")
        if url:
            urls.append(url)
    
    # Check if there's a "Next" button and click it to go to the next page
    next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    if next_button:
        next_button.click()
        time.sleep(3)  # Add a delay to allow the next page to load
    else:
        break  # Exit the loop if there is no "Next" button

# Print or process the collected URLs as needed
for url in urls:
    print(url)


https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTg0MDk3ODIzNjIxNjEwOjE2OTM3NTA0MDc6c3BfYXRmOjMwMDAzMTMyOTU5MDczMjo6MDo6&url=%2FKadence-A281BK-Professional-Acoustic-Rosewood%2Fdp%2FB0CD7D6C6R%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dguitar%26qid%3D1693750407%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTg0MDk3ODIzNjIxNjEwOjE2OTM3NTA0MDc6c3BfYXRmOjMwMDAyMjkzODAyNjEzMjo6MDo6&url=%2FKadence-Professional-Acoustic-Rosewood-Natural%2Fdp%2FB0CC927DLN%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dguitar%26qid%3D1693750407%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTg0MDk3ODIzNjIxNjEwOjE2OTM3NTA0MDc6c3BfYXRmOjMwMDAxNTUzNzk3OTIzMjo6MDo6&url=%2FKadence-Frontier-Acoustic-Guitar-Strings%2Fdp%2FB078GTJP5Y%2Fref%3Dsr_1_3_sspa%3Fkeywords%3Dguitar%26qid%3D1693750407%26sr%3D8-3-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo1NTg0MDk3ODIzN

In [88]:
len(urls)

168

In [89]:
# making empty list and fetching required data
brand_name = []
product_name = []
ratings = []
num_ratings = []
prices = []
exchange = []
exp_delivery = []
availability = []
other_details = []

for i in urls:
    driver.get(i)
    time.sleep(3)
    
    
    #fetching brand name 
    try:
        brand = driver.find_element(By.XPATH,"//a[@id='bylineInfo']")
        brand_name.append(brand.text)
    except NoSuchElementException:
        brand_name.append('-')
    
    
    # fetching Name of the Product
    try:
        product = driver.find_element(By.XPATH,"//span[@id='productTitle']")
        product_name.append(product.text)
    except NoSuchElementException:
        product_name.append('-')
        
        

     #fetching ratings
    try:
        rating = driver.find_element(By.XPATH,"//span[@class='a-size-base a-nowrap']/span")
        ratings.append(rating.text)
    except NoSuchElementException:
        ratings.append('-')
        
 
    #fetching  no of ratings
    try:
        num_rating = driver.find_element(By.XPATH,"//span[@id='acrCustomerReviewText']")
        num_ratings.append(num_rating.text)
    except NoSuchElementException:
        num_ratings.append('-')
        

    #fetching price of the product
    try:
        price = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')
        prices.append(price.text)
    except NoSuchElementException:
        prices.append('-')
        
        
    #fetching return/exchange
    try:
        exch = driver.find_element(By.XPATH,"(//span[@class='a-size-small a-color-link a-text-normal'])[2]")
        exchange.append(exch.text)
    except NoSuchElementException:
        exchange.append('-')
        

    #fetching expected delivery
    try:
        delivery = driver.find_element(By.XPATH,"//span[@class='a-text-bold']")
        exp_delivery.append(delivery.text)
    except NoSuchElementException:
        exp_delivery.append('-')
        

    #fetching availability information
    try:
        avail = driver.find_element(By.XPATH,"//span[@class='a-size-medium a-color-success']")
        availability.append(avail.text)
    except NoSuchElementException:
        availability.append('-')
        
        

In [90]:
print(len(brand_name),
len(product_name),
len(ratings),
len(num_ratings),
len(prices),
len(exchange),
len(exp_delivery),
len(availability),)


168 168 168 168 168 168 168 168


In [91]:
# Create the DataFrame
guitar = pd.DataFrame({
    'Brand Name': brand_name,
    'Name of the Product': product_name,
    'Rating': ratings,
    'No. of Ratings': num_ratings,
    'Price': prices,
    'Return/Exchange': exchange,
    'Expected Delivery': exp_delivery,
    'Availability': availability,
        'Product URL': urls
})

# Save the DataFrame to a CSV file
guitar.to_csv("Guitar_details.csv", index=False)

# Display the DataFrame
guitar


,Brand Name,Name of the Product,Rating,No. of Ratings,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Visit the Kadence Store,Kadence A281BK Professional Acoustic Rosewood ...,-,-,"6,599",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Visit the Kadence Store,Kadence A281 Professional Acoustic Rosewood gu...,5 out of 5,1 rating,"6,499",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the Kadence Store,"Kadence rosewood Guitar Frontier Series, Elect...",4 out of 5,"1,083 ratings","4,999",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Visit the Intern Store,"Intern INT-38C Right hand Acoustic Guitar Kit,...",4 out of 5,"13,412 ratings","1,899",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",4 out of 5,"20,203 ratings","1,999",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...
163,Brand: Martin Smith,Martin Smith W-101-BK-PK Acoustic Guitar Super...,3.8 out of 5,168 ratings,"5,349",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/Martin-Smith-W-101-BK-PK...
164,Visit the Intern Store,INTERN 41 inches Cutaway Shape Acoustic Guitar...,3.5 out of 5,9 ratings,"3,290",Pay on Delivery,"Thursday, 7 September",In stock,https://www.amazon.in/Acoustic-plectrums-stabi...
165,Visit the Fender Store,Fender SA105CE Electric Acoustic Guitar With M...,4.5 out of 5,4 ratings,"12,829",Pay on Delivery,"Friday, 8 September",In stock,https://www.amazon.in/Fender-SA105CE-Electric-...
166,Brand: NM Signature,Signature NM Blue Rosewood Fretboard Acoustic ...,-,-,"3,990",Pay on Delivery,"Wednesday, 13 September",-,https://www.amazon.in/Signature-NM-Rosewood-Fr...


# Q3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [51]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By

# Connecting to the webdriver
driver = webdriver.Chrome()

# Getting the webpage of the mentioned URL
url = "http://images.google.com/"
driver.get(url)

# Creating empty lists
urls = []

search_item = ["Fruits", "Cars", "Machine Learning", "Guitar", "Cakes"]

for item in search_item:
    driver.get(url)
    time.sleep(5)

    # Finding web element for the search bar
    search_bar = driver.find_element(By.NAME, "q")

    # Sending keys to get the keyword for the search bar
    search_bar.send_keys(str(item))

    # Clicking on the search button
    search_button = driver.find_element(By.XPATH, "//button[@type='submit']")
    search_button.click()

    # Scrolling down the webpage to get more images
    for _ in range(10):
        driver.execute_script("window.scrollBy(0,1000)")
        time.sleep(2)

    imgs = driver.find_elements(By.XPATH, "//img[@class='rg_i Q4LuWd']")
    
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None and source[:4] == 'http':
            img_url.append(source)
    
    for i in img_url[:10]:
        urls.append(i)

# the directory to save the images

save_dir ="C:/Users/somas/Downloads/downloaded_images/"

for i in range(len(urls)):
    if i >= 50:  # Limit the number of images to download
        break
    
    response = requests.get(urls[i])
    
    with open(save_dir + "image_" + str(i) + ".jpg", "wb") as file:
        file.write(response.content)

# Close the WebDriver
driver.quit()


In [117]:
# images are in downloaded_images Folder in github


# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV. 

In [112]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize WebDriver
driver = webdriver.Chrome()  # Replace with the path to your ChromeDriver

# Navigate to Flipkart's website
driver.get("https://www.flipkart.com/")

# Close the login pop-up if it appears (optional)
try:
    driver.find_element(By.XPATH, "//button[text()='✕']").click()
except:
    pass

# Search input
search_query = "Smart phones"  # Change this to your desired smartphone

# Enter the search query and submit
search_input = driver.find_element(By.NAME, "q")
search_input.send_keys(search_query)
search_input.submit()

# Wait for the search results to load
driver.implicitly_wait(10)

# Find all the product URLs on the first page
product_urls = []
product_links = driver.find_elements(By.XPATH, "//a[contains(@class,'_1fQZEK')]")
for link in product_links:
    product_url = link.get_attribute("href")
    product_urls.append(product_url)

# Close the WebDriver
#driver.quit()

# Display the list of product URLs
for url in product_urls:
    print(url)

https://www.flipkart.com/infinix-hot-20-play-aurora-green-64-gb/p/itm1e6903eb0203e?pid=MOBGK23WWFC8SUZZ&lid=LSTMOBGK23WWFC8SUZZTMFJG8&marketplace=FLIPKART&q=Smart+phones&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=c20bd9ef-232e-49a7-a2de-fba239de3c68.MOBGK23WWFC8SUZZ.SEARCH&ppt=hp&ppn=homepage&ssid=3tmcx8l3yo0000001693753614413&qH=4e3462c1e8f7cdf5
https://www.flipkart.com/infinix-hot-20-play-racing-black-64-gb/p/itm1e6903eb0203e?pid=MOBGK23WGFWJHRQD&lid=LSTMOBGK23WGFWJHRQD90DLPU&marketplace=FLIPKART&q=Smart+phones&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=c20bd9ef-232e-49a7-a2de-fba239de3c68.MOBGK23WGFWJHRQD.SEARCH&ppt=hp&ppn=homepage&ssid=3tmcx8l3yo0000001693753614413&qH=4e3462c1e8f7cdf5
https://www.flipkart.com/poco-m6-pro-5g-power-black-128-gb/p/itmef8fa46f89738?pid=MOBGRNZ3ER4N3K4F&lid=LSTMOBGRNZ3ER4N3K4FIYYGCU&marketplace=FLIPKART&q=Smart+phones&store=tyy%2F4io&srno=s_1_3&otracker=search&otracker1=search&fm=organic&ii

In [113]:
# Create empty lists to store the data
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
PsCamera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [114]:
#scraping the Brand_Name 
BName=driver.find_elements(By.XPATH,"//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)

#scraping the Storage_RAM_ROM 
ram=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)

#scraping the Camera 
PC=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        PsCamera.append("--") 
    else:
        PsCamera.append(i.text)
print(len(PsCamera),PsCamera)

#scraping the Display_size_Resolution 
DS=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)


#scraping the ProcessorAndCores 
P=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
print(len(ProcessorAndCores),ProcessorAndCores)

#scraping the Battery 
B=driver.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)


#scraping the Price 
price=driver.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)




24 ['13Mp + AI Lens | 8MP Front Camera', '13Mp + AI Lens | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '8MP Dual Rear Camera | 5MP Front Camera', '8MP Dual Rear Camera | 5MP Front Camera', '50MP + 8MP + 2MP | 8MP Front Camera', '13Mp + AI Lens | 8MP Front Camera', '50MP Dual Rear Camera | 5MP Front Camera', '8MP Dual Camera | 5MP Front Camera', '13Mp + AI Lens | 8MP Front Camera', '50MP Dual Rear Camera | 5MP Front Camera', '64MP + 2MP | 8MP Front Camera', '64MP + 2MP | 8MP Front Camera', '64MP + 2MP | 8MP Front Camera', '64MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '50MP Dual Rear Camera | 5MP Front Camera', '64MP Rear Camera | 16MP Front Camera', '13MP Rear Camera | 5MP Front Camera', '50MP + 8MP + 2MP | 8MP Front Camera', '50MP + 2MP | 8MP Front Camera', '8MP Dual Camera | 5MP Front Camera', '13MP + AI Lens | 5MP Front Camera', '13MP + AI Lens | 5MP Front Camera']
24 ['17.32 cm (6.82 inch) HD+ Display', '17.32 cm (6.82 inch) HD+ Display', '17.25 cm (6.79 

In [116]:
# Create the DataFrame
Flipkart = pd.DataFrame({
    'Brand_Name': Brand_Name,
    'Storage_RAM_ROM': Storage_RAM_ROM,
    'Amount P_F_Camera': PsCamera,
    'Display_size_Resolution': Display_size_Resolution,
    'ProcessorAndCores': ProcessorAndCores,
    'Battery': Battery,
    'Price': Price
})

Flipkart.to_csv("flipkart_mobies.csv",index = False)

Flipkart

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"Infinix HOT 20 Play (Aurora Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13Mp + AI Lens | 8MP Front Camera,17.32 cm (6.82 inch) HD+ Display,MediaTek G37 Processor,6000 mAh Li-ion Polymer Battery,"₹7,999"
1,"Infinix HOT 20 Play (Racing Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13Mp + AI Lens | 8MP Front Camera,17.32 cm (6.82 inch) HD+ Display,MediaTek G37 Processor,6000 mAh Li-ion Polymer Battery,"₹7,999"
2,"POCO M6 Pro 5G (Power Black, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 2MP | 8MP Front Camera,17.25 cm (6.79 inch) Full HD+ Display,Snapdragon 4 Gen 2 Processor,5000 mAh Battery,"₹12,999"
3,"POCO C51 (Power Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,8MP Dual Rear Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,Helio G36 Processor,5000 mAh Battery,"₹6,499"
4,"POCO C51 (Royal Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,8MP Dual Rear Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,Helio G36 Processor,5000 mAh Battery,"₹6,499"
5,"REDMI 12 (Moonstone Silver, 128 GB)",4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP + 8MP + 2MP | 8MP Front Camera,17.25 cm (6.79 inch) Full HD+ Display,Helio G88 Processor,5000 mAh Battery,"₹9,999"
6,"Infinix HOT 20 Play (Luna Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13Mp + AI Lens | 8MP Front Camera,17.32 cm (6.82 inch) HD+ Display,MediaTek G37 Processor,6000 mAh Li-ion Polymer Battery,"₹7,999"
7,"POCO C55 (Cool Blue, 128 GB)",6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,50MP Dual Rear Camera | 5MP Front Camera,17.04 cm (6.71 inch) HD+ Display,Mediatek Helio G85 Processor,5000 mAh Battery,"₹8,499"
8,"POCO C50 (Royal Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 512 GB,8MP Dual Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Mediatek Helio A22 Processor, Upto 2.0 GHz Pro...",5000 mAh Lithium Polymer Battery,"₹5,499"
9,"Infinix HOT 20 Play (Fantasy Purple, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 256 GB,13Mp + AI Lens | 8MP Front Camera,17.32 cm (6.82 inch) HD+ Display,MediaTek G37 Processor,6000 mAh Li-ion Polymer Battery,"₹7,999"


# Q5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Initialize the Chrome WebDriver (If its not in the same folder replace with the path to chromedriver executable)
driver = webdriver.Chrome()

# Open Google Maps
driver.get("https://www.google.com/maps")

# Wait for the Google Maps page to load
time.sleep(5)

# Find and interact with the search input element
search_box = driver.find_element(By.NAME,"q")  # The name attribute of the search box may change, inspect the page source for the current attribute name
city_name = input("Enter the city name: ")  # Input the city you want to search for
search_box.send_keys(city_name)
search_box.send_keys(Keys.RETURN)

# Wait for the search results to load
time.sleep(5)

# Extract the geospatial coordinates (latitude and longitude) from the URL
current_url = driver.current_url
coordinates = current_url.split('@')[1].split(',')[0:2]  # Extract latitude and longitude
latitude, longitude = coordinates

# Print the results
print(f"City: {city_name}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
print(f"URL: {current_url}")

# Close the browser
driver.quit()


Enter the city name: Hyderabad
City: Hyderabad
Latitude: 17.4127332
Longitude: 78.0783784
URL: https://www.google.com/maps/place/Hyderabad,+Telangana/@17.4127332,78.0783784,10z/data=!3m1!4b1!4m6!3m5!1s0x3bcb99daeaebd2c7:0xae93b78392bafbc2!8m2!3d17.385044!4d78.486671!16zL20vMDljNnc?entry=ttu


# Q6. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [52]:
# connecting to the webdriver
driver=webdriver.Chrome()
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)

In [53]:
# searching for best Laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(3)

In [54]:
# creating empty list
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [55]:
#scraping the data of laptop names
laptop_name = driver.find_elements(By.XPATH,"//span[@class='datahreflink']/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
#scraping the data of operating system
try:
    oper_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in oper_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#scraping data of display of the Laptop
try:
    display = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


# scraping data of processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass


# scraping the data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for m in memory:
        Memory.append(m.text)
except NoSuchElementException:
    pass


# scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


# scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


# scraping data of graph processor
try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details']//tr[5]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


# scraping the data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹ ','Rs'))
except NoSuchElementException:
    pass


In [56]:
print(len(Laptop_Name),
len(Operating_sys),
len(Display),
len(Processor),
len(Memory),
len(Weight),
len(Dimensions),
len(Graph_proc),
len(Price))

7 7 7 7 7 7 7 7 7


In [57]:
# Create a DataFrame
Gaming_Laptop = pd.DataFrame({
    'Laptop Name': Laptop_Name,
    'Operating System': Operating_sys,
    'Display': Display,
    'Processor': Processor,
    'Memory': Memory,
    'Weight': Weight,
    'Dimensions': Dimensions,
    'Graphical Processor': Graph_proc,
    'Price': Price
})

# Save the DataFrame to a CSV file
Gaming_Laptop.to_csv('gaming_laptops.csv', index=False)

In [58]:
Gaming_Laptop

,Laptop Name,Operating System,Display,Processor,Memory,Weight,Dimensions,Graphical Processor,Price
0,HP OMEN 17 (2023),Windows 11 Home,"17.3"" (2560 x 1440)",16 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,397.1 x 262 x 27 mm dimension & 2.78 kg weight,"₹ 269,777",16 GB DDR5GB RAM & 1 TB SSD,"Rs269,777"
1,MSI Titan GT77 12UHS,Windows 11 Home,"17.3"" (3840 x 2160)",64 GB DDR5GB RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,397 x 330 x 23 mm dimension & 3.3 kg weight,"₹ 499,990",64 GB DDR5GB RAM & 2 TB SSD,"Rs499,990"
2,Lenovo Legion 5i Pro,Windows 11 Home,"16"" (2560 x 1600)",32 GB DDR5GB RAM & 1 TB SSD,NVIDIA GeForce RTX 3070 Ti Graphics card,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,"₹ 179,990",32 GB DDR5GB RAM & 1 TB SSD,"Rs179,990"
3,ASUS ROG Strix Scar 18 2023,Windows 11 Home,"18"" (1920 x 1200)",32 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,294 x 399 x 23 mm dimension & 3.1 kg weight,"₹ 279,990",32 GB DDR5GB RAM & 1 TB SSD,"Rs279,990"
4,Acer Predator Helios Neo 16,Windows 11 Home,"16"" (2560 x 1600)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,360 x 279 x 28 mm dimension & 2.6 kg weight,"₹ 149,990",16 GB DDR5GB RAM & 1 TB SSD,"Rs149,990"
5,ASUS ROG Zephyrus G14,Windows 11 Home,"14"" (1920 x 1200)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 AMD Radeon RX 6700S Graphics card,312 x 227 x 19 mm dimension & 1.65 kg weight,"₹ 156,990",16 GB DDR5GB RAM & 1 TB SSD,"Rs156,990"
6,MSI Cyborg 15,Windows 11 Home,"15.6"" (1920 x 1080)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,& 1.98 kg weight,"₹ 125,000",16 GB DDR5GB RAM & 1 TB SSD,"Rs125,000"


# Q7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 

In [46]:
# Initialize WebDriver
driver = webdriver.Chrome()

# Navigate to the page with comments
driver.get("https://www.forbes.com/billionaires/")  # Replace with the actual URL
time.sleep(10)


All_Attribuites =[]

while True:
    
    # Iterate through the list in chunks of four elements
          
        # scraping the data of age of the billionaires
    total_tag = driver.find_elements(By.XPATH, '//div[@class="TableRow_cell__db-hv Table_cell__houv9"]')
    for i in total_tag:
        All_Attribuites.append(i.text)
    time.sleep(1)

    # clicking on next button
    try:
                
        next_button = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/button[2]/div')
        next_button.click()
    except:
        break




In [47]:
import pandas as pd

# creating empty lists
Rank = []
Person_Name = []
Net_worth = []
Age = []
Citizenship = []
Source = []
Industry = []

# Iterate through the list in chunks of 7 elements and creating their respective lists
for i in range(0, len(All_Attribuites), 7):
    Rank.append(All_Attribuites[i])
    Person_Name.append(All_Attribuites[i + 1])
    Net_worth.append(All_Attribuites[i + 2])
    Age.append(All_Attribuites[i + 3])
    Citizenship.append(All_Attribuites[i+4])
    Source.append(All_Attribuites[i+5])
    Industry.append(All_Attribuites[i+6])
    

In [48]:
print(len(Rank),
len(Person_Name),
len(Net_worth),
len(Age),
len(Citizenship),
len(Source),
len(Industry))

2640 2640 2640 2640 2640 2640 2640


In [50]:
# Create a dictionary from the lists
data_dict = {
    "Rank": Rank,
    "Person_Name": Person_Name,
    "Net_worth": Net_worth,
    "Age": Age,
    "Citizenship": Citizenship,
    "Source": Source,
    "Industry": Industry
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict)
print(df)
# Save the DataFrame to a CSV file
df.to_csv("forbies_billionaires_data.csv", index=False)

      Rank               Person_Name Net_worth Age    Citizenship  \
0        1  Bernard Arnault & family    $211 B  74         France   
1        2                 Elon Musk    $180 B  51  United States   
2        3                Jeff Bezos    $114 B  59  United States   
3        4             Larry Ellison    $107 B  78  United States   
4        5            Warren Buffett    $106 B  92  United States   
...    ...                       ...       ...  ..            ...   
2635  2540                   Yu Rong      $1 B  51          China   
2636  2540    Richard Yuengling, Jr.      $1 B  80  United States   
2637  2540             Zhang Gongyun      $1 B  60          China   
2638  2540    Zhang Guiping & family      $1 B  71          China   
2639  2540               Inigo Zobel      $1 B  66    Philippines   

                            Source               Industry  
0                             LVMH       Fashion & Retail  
1                    Tesla, SpaceX             Auto

In [59]:
df

,Rank,Person_Name,Net_worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,74,France,LVMH,Fashion & Retail
1,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$114 B,59,United States,Amazon,Technology
3,4,Larry Ellison,$107 B,78,United States,Oracle,Technology
4,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2635,2540,Yu Rong,$1 B,51,China,Health clinics,Healthcare
2636,2540,"Richard Yuengling, Jr.",$1 B,80,United States,Beer,Food & Beverage
2637,2540,Zhang Gongyun,$1 B,60,China,Tyre manufacturing machinery,Manufacturing
2638,2540,Zhang Guiping & family,$1 B,71,China,Real estate,Real Estate


# Q8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video. 

In [12]:
driver = webdriver.Chrome()

# opening the youtube.com
url = input("Enter The YouTube Video Url:")
driver.get(url)
time.sleep(2)


Enter The YouTube Video Url:https://www.youtube.com/watch?v=kKO1s-CUZvY


In [13]:
# 1000 times we scroll down by 10000 in order to generate more comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,10000)")

In [18]:
# creating empty lists
comments = []
comment_time = []
Likes = []
No_of_Likes = []

# scrape comments
cm = driver.find_elements(By.ID, "content-text")
for i in cm:
    if i.text is None:
        comments.append("--")
    else:
        comments.append(i.text)

# scrape time when comment was posted
tm = driver.find_elements(By.XPATH, "//a[contains(text(),'ago')]")


# scrape the comment likes
like = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in like:
    Likes.append(i.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
# Iterate through comments, time, and likes simultaneously
for i in range(len(cm)):
    if i < len(tm):
        comment_time.append(tm[i].text)
    else:
        comment_time.append("--")  # If there is no corresponding time, add a placeholder

    if i < len(like):
        Likes.append(like[i].text)
    else:
        Likes.append("--")  # If there is no corresponding like, add a placeholder

# Now, we have comments, corresponding times, and likes in synchronized lists


In [10]:
# Create a DataFrame
Youtube = pd.DataFrame({
    'Comment': comments[:500],
    'Comment Time': comment_time[:500],
    'Comment Upvotes': No_of_Likes[:500]
})
print(Youtube)

# Save the DataFrame to a CSV file
Youtube.to_csv('youtube_comments.csv', index=False)


                                               Comment           Comment Time  \
0    RIP to all those Armored Core fans who passed ...  8 months ago (edited)   
1    As a Titanfall fan, I had only ever heard of A...           7 months ago   
2     Fromsofts Art department is absolutely God-Tier.           7 months ago   
3    i love how the most replayed part is the BRRRR...           7 months ago   
4    Armored Core 6's announcement has been a treme...           8 months ago   
..                                                 ...                    ...   
495                     OST is gonna hit so fckng hard           8 months ago   
496  Is this basically a modern day of the old mech...           8 months ago   
497                              Entirely too cool lol           8 months ago   
498  I like that the Armored Core stories were alwa...           8 months ago   
499  Bro the Gundam mods for this game is gonna be ...           4 months ago   

    Comment Upvotes  
0    

In [60]:
Youtube

,Comment,Comment Time,Comment Upvotes
0,RIP to all those Armored Core fans who passed ...,8 months ago (edited),3.4K
1,"As a Titanfall fan, I had only ever heard of A...",7 months ago,847
2,Fromsofts Art department is absolutely God-Tier.,7 months ago,709
3,i love how the most replayed part is the BRRRR...,7 months ago,2.2K
4,Armored Core 6's announcement has been a treme...,8 months ago,875
...,...,...,...
495,OST is gonna hit so fckng hard,8 months ago,
496,Is this basically a modern day of the old mech...,8 months ago,
497,Entirely too cool lol,8 months ago,1
498,I like that the Armored Core stories were alwa...,8 months ago,30


# Q9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [131]:
# connecting to the webdriver
driver=webdriver.Chrome()

# getting the web page of mentioned url
url = "https://www.hostelworld.com/"
driver.get(url)
time.sleep(3)


In [132]:
# locating the location search bar
search_bar = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input')

# entering London in search bar
search_bar.send_keys("London")

In [133]:
# Find and click the button element by its XPath 
element = driver.find_element(By.XPATH, "//button[@class='item-content']")
element.click()

In [134]:
# Find and click the button element by its XPath
search = driver.find_element(By.XPATH, "//button[@class='btn-content large-button icon-only']")
search.click()

In [158]:
# creating empty list & find required data
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description = []
url = []

In [159]:
#House names
h_n= driver.find_elements(By.XPATH, '//div[@class="property-name"]/span')
for i in h_n:
    hostel_name.append(i.text)
print("Hostel Names:", hostel_name)


Hostel Names: ["Wombat's City Hostel London", "St Christopher's Village", 'Onefam Notting Hill', 'NX London Hostel', 'Urbany Hostel London', 'Generator London', 'Astor Hyde Park', 'Safestay London Elephant & Castle', 'Phoenix Hostel', 'Clink261', 'Astor Museum Inn', 'Smart Russell Square Hostel', 'Safestay London Kensington Holland Park', 'No.8 Willesden Hostel London', 'The Walrus Hostel', 'Destinations Hostels @ The Gallery', 'No.8 Seven Sisters', 'Queen Elizabeth Chelsea', 'Barmy Badger Backpackers', 'Astor Victoria', 'London Backpackers', "St Christopher's Hammersmith", 'Hostelle - women only hostel London', "St Christopher's Camden", 'YHA London Oxford Street', 'Kabannas London St Pancras', 'Prime Backpackers Angel', 'Bell House Hostel', 'Astor Kensington', 'Saint James Backpackers']


In [160]:
#Distance
dist= driver.find_elements(By.XPATH, "//span[@class='distance-description']")
for i in dist:
    distance.append(i.text)
print("Distances:", distance)


Distances: ['- 3.6km from city centre', '- 1.8km from city centre', '- 5.5km from city centre', '- 6.1km from city centre', '- 5.4km from city centre', '- 3km from city centre', '- 4.3km from city centre', '- 1.7km from city centre', '- 4.2km from city centre', '- 3.2km from city centre', '- 2.2km from city centre', '- 2.6km from city centre', '- 5.8km from city centre', '- 10km from city centre', '- 0.2km from city centre', '- 1.7km from city centre', '- 9km from city centre', '- 5.7km from city centre', '- 5.5km from city centre', '- 1.8km from city centre', '- 11.9km from city centre', '- 7.5km from city centre', '- 5.1km from city centre', '- 4.3km from city centre', '- 2.1km from city centre', '- 3.3km from city centre', '- 3.6km from city centre', '- 4.2km from city centre', '- 4.9km from city centre', '- 5.5km from city centre']


In [161]:
# Ratings
rate = driver.find_elements(By.XPATH,' //span[@class="left-margin"]')
for i in rate:
    rating.append(i.text)
print("Ratings:", rating)


Ratings: ['(15265)', '(12379)', '(2233)', '(2093)', '(895)', '(7714)', '(12162)', '(5093)', '(4386)', '(385)', '(9326)', '(10081)', '(1637)', '(5079)', '(4191)', '(221)', '(4054)', '(3548)', '(1971)', '(15015)', '(4510)', '(4295)', '(28)', '(4087)', '(4652)', '(2887)', '(789)', '(92)', '(6531)', '(1918)']


In [162]:
#Overall_reviews
o_r = driver.find_elements(By.XPATH,'//span[@class="keyword"]')
for i in o_r:
    over_all.append(i.text)
over_all.pop(0)
print("Overall Reviews:", over_all)

Overall Reviews: ['Fabulous', 'Superb', 'Fabulous', 'Superb', 'Fabulous', 'Superb', 'Very Good', 'Fabulous', 'Very Good', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good', 'Very Good', 'Good', 'Fabulous', 'Superb', 'Very Good', 'Superb', 'Very Good', 'Very Good', 'Very Good', 'Fabulous', 'Very Good', 'Fabulous', 'Fabulous', 'Very Good', 'Very Good', 'Fabulous', 'Very Good']


In [164]:
# Facilities
fac = driver.find_elements(By.XPATH, ' //div[@class="badge-wrapper theme-blue-light solo-icon"]')
for i in fac:
    facilities.append(i.text)
print("Facilities:", facilities)

Facilities: ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [165]:
print(len(hostel_name),len(distance),len(rating),len(over_all))

30 30 30 30


In [175]:
data = {
        'Hostel Name': hostel_name,
        'Distance': distance,
        'Rating': rating,
        'Overall Rating': over_all
    }

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('hostel_data.csv', index=False)

In [176]:
df

,Hostel Name,Distance,Rating,Overall Rating
0,Wombat's City Hostel London,- 3.6km from city centre,(15265),Fabulous
1,St Christopher's Village,- 1.8km from city centre,(12379),Superb
2,Onefam Notting Hill,- 5.5km from city centre,(2233),Fabulous
3,NX London Hostel,- 6.1km from city centre,(2093),Superb
4,Urbany Hostel London,- 5.4km from city centre,(895),Fabulous
5,Generator London,- 3km from city centre,(7714),Superb
6,Astor Hyde Park,- 4.3km from city centre,(12162),Very Good
7,Safestay London Elephant & Castle,- 1.7km from city centre,(5093),Fabulous
8,Phoenix Hostel,- 4.2km from city centre,(4386),Very Good
9,Clink261,- 3.2km from city centre,(385),Very Good
